📝 **Author:** Amirhossein Heydari - 📧 **Email:** amirhosseinheydari78@gmail.com - 📍 **Linktree:** [linktr.ee/mr_pylin](https://linktr.ee/mr_pylin)

---

# Dependencies

In [1]:
from collections import Counter

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary

from datasets import load_dataset

In [2]:
# set a seed for deterministic results
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
# check if cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# log
device

device(type='cuda')

# Word Embedding using Word2Vec [Skip-Grams method]
🌟 **Example**:
   - Sentence: **The quick brown fox jumps over the lazy dog.**
   - **WINDOW_SIZE: 2**
      - For "The" (center word), context words are ["quick"].
      - For "quick" (center word), context words are ["The", "brown"].
      - For "brown" (center word), context words are ["quick", "The", "fox", "jumps"].
      - For "fox" (center word), context words are ["brown", "quick", "jumps"].
      - For "jumps" (center word), context words are ["fox", "brown", "over"].
      - For "over" (center word), context words are ["jumps", "the"].
      - For "the" (center word), context words are ["over"].
   - **NEGATIVE_SAMPLES: 5**
      - If we are predicting "brown" and the context words are ["quick", "The", "fox", "jumps"].
      - we will also randomly select 5 words from the rest of the vocabulary (e.g., "lazy", "dog", etc.) and treat them as negative samples.

## Hyperparameters

In [4]:
EMBEDDING_DIM = 100
WINDOW_SIZE = 2
BATCH_SIZE = 2048
EPOCHS = 2
LEARNING_RATE = 0.01
NEGATIVE_SAMPLES = 5

## Step 1: Load and preprocess the dataset

In [5]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", cache_dir="../../datasets/")
train_text = dataset["train"]["text"]

In [6]:
# tokenize and build vocabulary
tokenized_sentences = [sentence.split() for sentence in train_text if sentence.strip()]
words = [word for sentence in tokenized_sentences for word in sentence]
vocab = Counter(words)

# log
print(f"vocabulary size             : {len(vocab)}")
print(f"top 5 most  frequent tokens : {vocab.most_common(5)}")
print(f"top 5 least frequent tokens : {sorted(vocab.items(), key=lambda x: x[1])[:5]}")

vocabulary size             : 76616
top 5 most  frequent tokens : [('the', 113161), (',', 99913), ('.', 73388), ('of', 56889), ('and', 50603)]
top 5 least frequent tokens : [('Unrecorded', 1), ('Calamaty', 1), ('forgiving', 1), ('unvoiced', 1), ('scanned', 1)]


In [7]:
# filter out rare words
min_freq = 5
vocab = {word: freq for word, freq in vocab.items() if freq >= min_freq}
word2idx = {word: idx for idx, (word, _) in enumerate(vocab.items())}
idx2word = {idx: word for word, idx in word2idx.items()}
vocab_size = len(word2idx)

# log
print(f"vocabulary size             : {vocab_size}")
print(f"top 5 most  frequent tokens : {sorted(vocab.items(), key=lambda x: x[1], reverse=True)[:5]}")
print(f"top 5 least frequent tokens : {sorted(vocab.items(), key=lambda x: x[1])[:5]}")

vocabulary size             : 23378
top 5 most  frequent tokens : [('the', 113161), (',', 99913), ('.', 73388), ('of', 56889), ('and', 50603)]
top 5 least frequent tokens : [('Senjō', 5), ('penal', 5), ('Gallia', 5), ('adjustments', 5), ('newcomers', 5)]


In [8]:
# generate skip-grams with negative sampling
def generate_skip_grams(words: list[str], word2idx: dict[str, int], window_size: int, vocab: dict[str, int]) -> list[tuple[int, int]]:
    pairs = []
    for idx, word in enumerate(words):
        if word not in word2idx:
            continue
        center_idx = word2idx[word]
        context_range = range(max(0, idx - window_size), min(len(words), idx + window_size + 1))
        for context_idx in context_range:
            if context_idx == idx:
                continue
            context_word = words[context_idx]
            if context_word in word2idx:
                pairs.append((center_idx, word2idx[context_word]))
    return pairs

# generate skip-grams
skip_grams = generate_skip_grams(words, word2idx, WINDOW_SIZE, vocab)

# log
print(f"total number of skip-grams         : {len(skip_grams)}")
print(f"sequence of words [1000 to 1006]   : {words[1000:1007]}")
print("{word:idx} for the above sequence] :", {w: word2idx[w] for w in words[1000:1007]})
print(f"skip-gram pairs for above sequence : {[skip_grams[skip_grams.index((401, 200)) + i] for i in range(WINDOW_SIZE)]}")

total number of skip-grams         : 7532842
sequence of words [1000 to 1006]   : ['they', 'are', 'nevertheless', 'up', 'to', 'the', 'task']
{word:idx} for the above sequence] : {'they': 200, 'are': 72, 'nevertheless': 401, 'up': 402, 'to': 18, 'the': 14, 'task': 403}
skip-gram pairs for above sequence : [(401, 200), (401, 72)]


## Step 2: Create Dataset and DataLoader

In [9]:
class SkipGramDataset(Dataset):
    def __init__(self, skip_grams, vocab_size, neg_samples):
        self.skip_grams = skip_grams
        self.vocab_size = vocab_size
        self.neg_samples = neg_samples
    
    def __len__(self):
        return len(self.skip_grams)
    
    def __getitem__(self, idx):
        center, context = self.skip_grams[idx]
        negatives = torch.multinomial(
            torch.tensor([1.0] * self.vocab_size), self.neg_samples, replacement=True
        )
        return torch.tensor(center, dtype=torch.long), torch.tensor(context, dtype=torch.long), negatives

In [10]:
dataset = SkipGramDataset(skip_grams, vocab_size, NEGATIVE_SAMPLES)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
num_batches = len(data_loader)

# log
for i in range(WINDOW_SIZE * 2):
    temp_sample = dataset[skip_grams.index((401, 200)) + i]
    print(temp_sample)
    print(f"  negative samples:")
    for j in range(NEGATIVE_SAMPLES):
        print("    ", {temp_sample[2][j].item(): idx2word[temp_sample[2][j].item()]})
    print()

(tensor(401), tensor(200), tensor([ 1359,  1470,  2889,  1229, 12300]))
  negative samples:
     {1359: '520'}
     {1470: 'medicines'}
     {2889: 'Vancouver'}
     {1229: 'ultimate'}
     {12300: 'Palestine'}

(tensor(401), tensor(72), tensor([11146, 22331, 21712,  1953,  3100]))
  negative samples:
     {11146: 'antagonist'}
     {22331: 'Rajamouli'}
     {21712: 'Honors'}
     {1953: 'show'}
     {3100: 'statistic'}

(tensor(401), tensor(402), tensor([ 3671,  8775, 19696, 20350,  8825]))
  negative samples:
     {3671: 'disguise'}
     {8775: 'Influences'}
     {19696: 'Store'}
     {20350: 'Novoselic'}
     {8825: 'arch'}

(tensor(401), tensor(18), tensor([14317,  2059, 16392, 14573, 10222]))
  negative samples:
     {14317: 'Fraser'}
     {2059: 'focusing'}
     {16392: 'concentric'}
     {14573: 'publishes'}
     {10222: 'lifestyle'}



## Step 3: Define and Initialize the Word2Vec model

In [11]:
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.context_embeddings = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, center, context, negatives):
        center_embed = self.embeddings(center)
        context_embed = self.context_embeddings(context)
        neg_embed = self.context_embeddings(negatives)

        # positive scores
        positive_score = torch.sum(center_embed * context_embed, dim=1)
        positive_loss = -torch.log(torch.sigmoid(positive_score + 1e-9))

        # negative scores
        negative_score = torch.bmm(neg_embed, center_embed.unsqueeze(2)).squeeze()
        negative_loss = -torch.sum(torch.log(torch.sigmoid(-negative_score + 1e-9)), dim=1)

        return (positive_loss + negative_loss).mean()

In [12]:
model = Word2Vec(vocab_size, EMBEDDING_DIM).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [13]:
model

Word2Vec(
  (embeddings): Embedding(23378, 100)
  (context_embeddings): Embedding(23378, 100)
)

In [14]:
summary(
    model,
    input_data=(
        (
            torch.randint(0, vocab_size, (BATCH_SIZE,)).to(device),
            torch.randint(0, vocab_size, (BATCH_SIZE,)).to(device),
            torch.randint(0, vocab_size, (BATCH_SIZE, NEGATIVE_SAMPLES)).to(device)
        )
    ),
)

Layer (type:depth-idx)                   Output Shape              Param #
Word2Vec                                 --                        --
├─Embedding: 1-1                         [2048, 100]               2,337,800
├─Embedding: 1-2                         [2048, 100]               2,337,800
├─Embedding: 1-3                         [2048, 5, 100]            (recursive)
Total params: 4,675,600
Trainable params: 4,675,600
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 14.36
Input size (MB): 0.11
Forward/backward pass size (MB): 11.47
Params size (MB): 18.70
Estimated Total Size (MB): 30.29

## Step 5: Training Loop

In [15]:
for epoch in range(EPOCHS):
    total_loss = 0
    model.train()
    
    # log
    print(f"epoch {epoch + 1:>0{len(str(EPOCHS))}}/{EPOCHS}")
    
    for i, (center, context, negatives) in enumerate(data_loader):
        # Move data to the correct device
        center, context, negatives = center.to(device), context.to(device), negatives.to(device)
        
        optimizer.zero_grad()
        loss = model(center, context, negatives)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # log
        if i % 100 == 0:
            print(f"  iteration {i + 1:>0{len(str(num_batches))}}/{num_batches}  |  loss: {loss.item():.4f}  |  total loss: {total_loss:.4f}")
    
    # log
    print(f"epoch {epoch + 1:>0{len(str(EPOCHS))}}/{EPOCHS}  |  total loss: {total_loss:.4f}")
    print('-' * 50)

epoch 1/2
  iteration 0001/3679  |  loss: 24.5571  |  total loss: 24.5571
  iteration 0101/3679  |  loss: 16.3418  |  total loss: 2042.5032
  iteration 0201/3679  |  loss: 11.6497  |  total loss: 3448.2477
  iteration 0301/3679  |  loss: 9.0296  |  total loss: 4465.6896
  iteration 0401/3679  |  loss: 6.8504  |  total loss: 5235.8727
  iteration 0501/3679  |  loss: 5.2581  |  total loss: 5843.1700
  iteration 0601/3679  |  loss: 4.7532  |  total loss: 6337.2399
  iteration 0701/3679  |  loss: 3.7600  |  total loss: 6754.2973
  iteration 0801/3679  |  loss: 3.4022  |  total loss: 7118.3565
  iteration 0901/3679  |  loss: 3.0965  |  total loss: 7442.5429
  iteration 1001/3679  |  loss: 2.8195  |  total loss: 7738.0126
  iteration 1101/3679  |  loss: 2.5631  |  total loss: 8013.6600
  iteration 1201/3679  |  loss: 2.6182  |  total loss: 8273.9914
  iteration 1301/3679  |  loss: 2.2488  |  total loss: 8521.8611
  iteration 1401/3679  |  loss: 2.3285  |  total loss: 8759.4935
  iteration 15

## Step 6: Save embeddings

In [ ]:
embeddings = model.embeddings.weight.data
torch.save(embeddings, "../../assets/embeddings/word2vec_embeddings.pt")

# log
print()

## Step 7: Visualize word embeddings using t-SNE

In [20]:
def visualize_embeddings(embeddings: torch.Tensor, idx2word: dict[int, str], num_points: int = 100) -> None:
    tsne = TSNE(n_components=2, random_state=seed)
    reduced_embeddings = tsne.fit_transform(embeddings[:num_points].cpu().numpy())
    
    plt.figure(figsize=(10, 10))
    for idx, (x, y) in enumerate(reduced_embeddings):
        plt.scatter(x, y)
        plt.text(x, y, idx2word[idx], fontsize=9)
    plt.show()

In [ ]:
visualize_embeddings(embeddings, idx2word)